# Read WWW for various car data
and create lqlite3 database

In [153]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3
import re
from datetime import datetime, timedelta
from collections import OrderedDict

strCarSalesQ=r'https://auto.vercity.ru/statistics/sales/europe/{0}/russia/01-12/'
strCarProdsQ=r'https://auto.vercity.ru/statistics/production/europe/{0}/russia/01-12/'

conn=sqlite3.connect('cars.sqlite3') #connect or create database


In [133]:
curYear=2010

def get_date_list(iYear):
        dates=['{0}-01-01'.format(iYear), '{0}-01-01'.format(iYear+1)]
        start, end=[datetime.strptime(_, '%Y-%m-%d') for _ in dates]
        od=OrderedDict(((start+timedelta(_)).strftime(r'%b-%Y'), None) for _ in range((end-start).days))
        return list(od.keys())

def GetYearsSalesData(curYear, strSource=strCarSalesQ):

    r=requests.get(strSource.format(curYear))
    soup=BeautifulSoup(r.text, 'html.parser')
    
    lstHeads=['Model']+get_date_list(curYear)
    pdf=pd.DataFrame(columns=lstHeads)
    #print(pdf)

    lstVals=[]
    lstI=[1]+list(range(3, 15)) #+ [16] 

    for dv in soup.findAll('div', class_='pages_table-scrollable'):
        t=dv.table
        for tr in t.find_all('tr'):
            th=tr.find_all('th')
            if len(th)==0:
                tds=tr.find_all('td')
                if re.sub('\s', '', tds[1].text) =='N/A':
                    break
                lstVals=[re.sub('\s', '', c.text) for i, c in enumerate(tds) if i in lstI]
                pdf=pdf.append(pd.Series(lstVals, index=lstHeads), ignore_index=True)
    return pdf.set_index('Model')

#print(GetYearsSalesData(2010).stack())


## 1. Car sales (https://auto.vercity.ru/statistics/sales/)
###  For Russia

https://auto.vercity.ru/statistics/sales/europe/2010/russia/01-12/

In [144]:
#read www-data and make dataframe
sales_df=pd.concat([GetYearsSalesData(Y) for Y in range(2010, 2019)], axis=1) 

sales_df=sales_df.stack().reset_index().rename(columns={'level_0':'Model', 'level_1':'date', 0:'Count'})
sales_df['Count']=sales_df['Count'].apply(pd.to_numeric)

print(sales_df.head(10))



/home/egor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


      Model      date  Count
0     Acura  Jan-2010      0
1     Acura  Feb-2010      0
2     Acura  Mar-2010      0
3     Acura  Apr-2010      0
4     Acura  May-2010      0
5     Acura  Jun-2010      0
6     Acura  Jul-2010      0
7     Acura  Aug-2010      0
8     Acura  Sep-2010      0
9     Acura  Oct-2010      0
10    Acura  Nov-2010      0
11    Acura  Dec-2010      0
12    Acura  Jan-2011      0
13    Acura  Feb-2011      0
14    Acura  Mar-2011      0
15    Acura  Apr-2011      0
16    Acura  May-2011      0
17    Acura  Jun-2011      0
18    Acura  Jul-2011      0
19    Acura  Aug-2011      0
20    Acura  Sep-2011      0
21    Acura  Oct-2011      0
22    Acura  Nov-2011      0
23    Acura  Dec-2011      0
24    Acura  Jan-2012      0
25    Acura  Feb-2012      0
26    Acura  Mar-2012      0
27    Acura  Apr-2012      0
28    Acura  May-2012      0
29    Acura  Jun-2012      0
...     ...       ...    ...
7158    УАЗ  Jul-2016   3290
7159    УАЗ  Aug-2016   4161
7160    УАЗ  S

In [150]:
sales_df=sales_df[sales_df['Count']!=0]

sales_df.to_sql('sales_ru', con=conn, if_exists='replace', index=False)


## 2. Car production (https://auto.vercity.ru/statistics/production/)
###  For Russia

https://auto.vercity.ru/statistics/production/europe/2018/russia/

In [154]:
prod_df=pd.concat([GetYearsSalesData(Y, strSource=strCarProdsQ) for Y in range(2010, 2019)], axis=1) 

prod_df=prod_df.stack().reset_index().rename(columns={'level_0':'Model', 'level_1':'date', 0:'Count'})
prod_df['Count']=prod_df['Count'].apply(pd.to_numeric)

print(prod_df.head(10))

/home/egor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


  Model      date  Count
0   BAW  Jan-2010      0
1   BAW  Feb-2010      0
2   BAW  Mar-2010      0
3   BAW  Apr-2010      0
4   BAW  May-2010      0
5   BAW  Jun-2010      0
6   BAW  Jul-2010      0
7   BAW  Aug-2010      0
8   BAW  Sep-2010      0
9   BAW  Oct-2010      0


In [156]:
prod_df=prod_df[prod_df['Count']!=0]

prod_df.to_sql('production_ru', con=conn, if_exists='replace', index=False)

In [159]:
print(prod_df[prod_df['Model']=='Opel'])

     Model      date  Count
3744  Opel  Jan-2010    576
3745  Opel  Feb-2010    252
3746  Opel  Mar-2010    837
3747  Opel  Apr-2010   1212
3748  Opel  May-2010   1393
3749  Opel  Jun-2010   1816
3750  Opel  Jul-2010   2404
3751  Opel  Aug-2010   2671
3752  Opel  Sep-2010   2841
3753  Opel  Oct-2010   2520
3754  Opel  Nov-2010   2401
3755  Opel  Dec-2010   2970
3756  Opel  Jan-2011   1656
3757  Opel  Feb-2011   2235
3758  Opel  Mar-2011   2547
3759  Opel  Apr-2011   2520
3760  Opel  May-2011   2379
3761  Opel  Jun-2011   2600
3762  Opel  Jul-2011   2616
3763  Opel  Aug-2011   3028
3764  Opel  Sep-2011   3800
3765  Opel  Oct-2011   4228
3766  Opel  Nov-2011   3619
3767  Opel  Dec-2011   3500
3768  Opel  Jan-2012   2103
3769  Opel  Feb-2012   6463
3770  Opel  Mar-2012   5790
3771  Opel  Apr-2012   4803
3772  Opel  May-2012   4460
3773  Opel  Jun-2012   4623
...    ...       ...    ...
3776  Opel  Sep-2012   4017
3777  Opel  Oct-2012   4983
3778  Opel  Nov-2012   5476
3779  Opel  Dec-2012